In [1]:
from data_classes import LAQNData, HealthData, MetData, IncomeData
from os import path
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import requests

In [2]:
species = "NO2"
region = "London"
NO2 = LAQNData(species, region)

In [3]:
income = IncomeData()

Load the resampled daily NO2 data:

In [ ]:
daily_NO2_df = NO2.read("laqn-resample")
daily_NO2_df.head(5)

Load the resampled daily income data:

In [ ]:
daily_income_df = income.read("income-resample")
daily_income_df.head(5)

Load the LAQN metdata:

In [ ]:
meta_url = "http://api.erg.kcl.ac.uk/AirQuality/Information/MonitoringSites/GroupName=London/Json"
sites_request = requests.get(meta_url)
meta_df = pd.DataFrame(sites_request.json()['Sites']['Site'])
meta_df.head(5)

Rename the local authorities in the income metadata using the LAQN metadata:

In [8]:
# income_metadata_df = income.rename_local_authority_districts_and_log(reference="use_LAQN")

Load the renamed income metadata:

In [6]:
income_metadata_df = income.read("income-metadata")

wandb: wandb version 0.12.17 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
